# Proyecto 2
Todas las preguntas tienen el mismo peso en esta tarea. Esta tarea requiere más aprendizaje individual. Te invito a consultar la [documentación de pandas](https://pandas.pydata.org/pandas-docs/stable/) para encontrar funciones o métodos que quizás no has usado aún, o realices preguntas en [Stack Overflow](https://stackoverflow.com/) y etiquételos como relacionados con pandas y python. Todas las preguntas tienen la misma cantidad de puntos, excepto la pregunta 1, que vale el 20 % de la calificación de la tarea.

**Nota**: A partir de la pregunta 2 se basan en la respuesta de la pregunta 1.

In [1]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings

warnings.filterwarnings('ignore')

### Pregunta 1
Cargue los datos de energía del archivo `assets/Energy Indicators.xls`, que es una lista de indicadores de [suministro de energía y producción de electricidad renovable](assets/Energy%20Indicators.xls) de las [Naciones Unidas](https://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) para el año 2013, y asigna esos datos a un DataFrame con el nombre de variable **energy**.

Ten en cuenta que este es un archivo de Excel y no un archivo de valores separados por comas (.csv). Además, asegúrate de excluir la información del pie de página y del encabezado del archivo de datos. Las primeras dos columnas son innecesarias, por lo que debes deshacerse de ellas. También debes cambiar las etiquetas de las columnas para que las columnas sean las siguientes:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convierta `Energy Supply` a gigajulios (**Nota: hay 1,000,000 giga julios en un petajulio**). Para todos los países que carezcan de datos (por ejemplo, datos con "..."), asegúrese de que estos reflejen valores `np.NaN`.

Cambia el nombre de los siguientes países según este listado (nos servirá para preguntas posteriores):

```
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

También hay varios países que contienen paréntesis en su nombre. Asegúrete de eliminarlos, por ejemplo: `'Bolivia (Estado Plurinacional de)'` debería ser `'Bolivia'`.

A continuación, carga los datos del GDP (PBI) del archivo `assets/world_bank.csv`. Este es un archivo .csv del [Banco Mundial](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD) que contiene el PBI de los países desde 1960 hasta 2015. Llame a este DataFrame **gdp**.

Asegúrete de omitir el encabezado y cambia el nombre de la siguiente lista de países:

```
"Korea, Rep.": "South Korea",
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finalmente, lee el [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](https://www.scimagojr.com/countryrank.php?category=2102) desde el archivo `assets/scimagojr-3.xlsx`, que clasifica a los paises según sus contribuciones al área mencionada. Denomina este DataFrame con el nombre **sciamgo**.

Unifica los tres datasets: GDP, Energy, y ScimEn en un nuevo dataset (utilizando la intersección de los nombres de países). Utiliza solo los años del 2006 al 2015 de datos de GDP (PBI) y solo los 15 principales países según el ranking de Scimagojr 'Rank' (del 1 al 15).

El índice de este DataFrame debe ser el nombre de los países, y las columnas deben ser ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

*Esto debería devolver un DataFrame con 20 columnas y 15 entradas, y las filas del DataFrame deberían estar ordenadas por "Rank".*


In [7]:
import pandas as pd
import numpy as np
 # Cargar los datos de Energy Indicators.xls
energy = pd.read_excel('assets/Energy Indicators.xls', skiprows=17, skipfooter=38)
energy = energy.iloc[:, 2:] #Sirve para no tomar en cuenta las 2 primeras columnas
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x*1000000 if x != '...' else np.nan)
energy['Energy Supply per Capita'] = energy['Energy Supply per Capita'].apply(lambda x: x if x != '...' else np.nan)
energy['Country'] = energy['Country'].str.replace(r"\s*\(.*\)", "", regex=True)  # Eliminar paréntesis y su contenido
energy['Country'] = energy['Country'].str.replace(r"\d+$", "", regex=True)
energy['Country'] = energy['Country'].str.strip()
energy['Country'].replace({
    #"China2": "China",
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    #"Japan10": "Japan",
    #"France6" : "France",
    #"Italy9": "Italy",
    #"Spain16": "Spain",
    #"Australia1" : "Australia",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}, inplace=True)


# Cargar los datos del GDP (PBI) del archivo world_bank.csv
gdp = pd.read_csv('assets/world_bank.csv', skiprows=4)
gdp['Country Name'].replace({
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}, inplace=True)

# Cargar los datos del Scimago Journal and Country Rank
scimgo = pd.read_excel('assets/scimagojr-3.xlsx')


    # Seleccionar solo los 15 principales países según el ranking de Scimgo
scimgo = scimgo[scimgo['Rank'] <= 15]

def load_data():

    # Unificar los tres DataFrames
    combined = pd.merge(scimgo, energy, how='inner', left_on='Country', right_on='Country')

    combined = pd.merge(combined, gdp, how='inner', left_on='Country', right_on='Country Name')

    # Seleccionar las columnas necesarias y establecer el índice
    columns = ['Country', 'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
            'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
            '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    combined = combined[columns]
    combined.set_index('Country', inplace=True)

    # Mostrar el DataFrame resultante
    return(combined)

load_data()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Pregunta 2
La pregunta anterior unió tres datasets y luego los redujo a las 15 entradas principales. Pero antes de reducirlos a los 15 elementos principales, ¿cuántas entradas se perdieron?

*Esta función debería devolver un solo número.*

In [1]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Todo menos esto!</text>
</svg>

In [8]:
def outer():    
    num_energy = len(energy)
    num_gdp = len(gdp)
    num_scimgo = len(scimgo)

    merged = pd.merge(scimgo, energy, how='outer', left_on='Country', right_on='Country')

    merged = pd.merge(merged, gdp, how='outer', left_on='Country', right_on='Country Name')

    num_merged = len(merged)
    num_combined = len (load_data())
    result = num_merged - num_combined
    return result
outer()

290

### Pregunta 3
¿Cuáles son los 15 países con mayor GDP (PBI) promedio en los últimos 10 años?

*Esta función debería devolver una serie llamada `avgGDP` con 15 países y su GDP (PBI) promedio ordenado en orden descendente.*

In [9]:
def gdp_top_avg():
    gdp_columns = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    gdp_data = gdp[gdp_columns]

    avgGDP_ = gdp_data.mean(axis=1)
    avgGDP_with_names = gdp[['Country Name']].copy()
    avgGDP_with_names['Average GDP'] = avgGDP_

    # Ordena los países por su promedio en orden descendente
    avgGDP_with_names = avgGDP_with_names.sort_values(by='Average GDP', ascending=False)
    
    top_15_countries = avgGDP_with_names.head(15)
    avgGDP = top_15_countries.set_index('Country Name')['Average GDP']

    return avgGDP

gdp_top_avg()

Country Name
World                        6.713548e+13
High income                  4.586468e+13
OECD members                 4.514723e+13
Post-demographic dividend    4.292131e+13
IDA & IBRD total             2.202600e+13
Europe & Central Asia        2.117122e+13
Low & middle income          2.081219e+13
Middle income                2.051324e+13
IBRD only                    2.039738e+13
European Union               1.717367e+13
East Asia & Pacific          1.714490e+13
North America                1.703064e+13
Upper middle income          1.576611e+13
United States                1.536434e+13
Late-demographic dividend    1.389504e+13
Name: Average GDP, dtype: float64

### Pregunta 4
¿Cuánto cambió el GDP (PBI) durante el lapso de 10 años para el país con el sexto GDP (PBI) promedio más grande?

*Esta función debería devolver un solo número.*

In [10]:
def gdp_changes():
    sixth_country = gdp_top_avg().index[5]
    gdp_2006 = gdp[gdp['Country Name'] == sixth_country]['2006'].values[0]
    gdp_2015 = gdp[gdp['Country Name'] == sixth_country]['2015'].values[0]
    gdp_change = gdp_2015 - gdp_2006
   
    return gdp_change
gdp_changes()

1787625194095.8008

### Pregunta 5
¿Cuál es el suministro medio de energía per cápita (`Energy Supply per Capita`)?

*Esta función debería devolver un solo número.*

In [11]:
def energy_per_capita():
    avg_energy_per_capita = load_data()['Energy Supply per Capita'].mean()
    return avg_energy_per_capita
energy_per_capita()

157.6

### Pregunta 6
¿Qué país tiene el máximo `% Renewable` y cuál es el porcentaje?

*Esta función debe devolver una tupla con el nombre del país y el porcentaje.*

In [12]:
def max_renewable():
    max_ren = load_data()['% Renewable'].idxmax()
    max_ren_percentage = load_data().loc[max_ren, '% Renewable']
    return (max_ren, max_ren_percentage)
max_renewable()

('Brazil', 69.64803)

### Pregunta 7
Cree una nueva columna que sea la relación entre `Self-Citations` y `Total Citations`.
¿Cuál es el valor máximo para esta nueva columna y qué país tiene la relación más alta?

*Esta función debe devolver una tupla con el nombre del país y el ratio (proporción).*

In [13]:
def self_citations():
    combined = load_data()
    combined['Citations Ratio'] = combined['Self-citations'] / combined['Citations']
    max_id =  combined['Citations Ratio'].idxmax()
    max = combined.loc[max_id, 'Citations Ratio']
    return (max_id, max)

self_citations()

('China', 0.6893126179389422)

### Pregunta 8

Crea una columna que estime la población utilizando `Energy Supply` y `Energy Supply per capita`.
¿Cuál es el tercer país más poblado según esta estimación?

*Esta función debería devolver el nombre del país*

In [14]:
def population():
    data = load_data()
    data['Population'] = data['Energy Supply'] / data['Energy Supply per Capita']
    data = data.sort_values(by = 'Population', ascending= False)
    thrid = data.iloc[2].name
    return thrid
population()

'United States'

### Pregunta 9
Crea una columna que estime el número de `Citable documents` por persona.
¿Cuál es la correlación entre el número de `Citable documents` per cápita y el `Energy Supply per Capita`? Utiliza el método `.corr()`, (correlación de Pearson).

*Esta función debería devolver un solo número.*

*(Opcional: utiliza la función `plot9()` para visualizar la relación entre el suministro de energía per cápita y los `Citable documents` per cápita)*

In [15]:
def citable_documents_per_capita():
    data = load_data()
    data['Citable documents per capita'] = data['Citable documents']  / data['Energy Supply per Capita']
    correlacion = data['Citable documents'].corr(data['Energy Supply per Capita'])
    return correlacion
citable_documents_per_capita()

0.10342386043436658

In [2]:
def plot9():
    import matplotlib as plt
    %matplotlib inline

    top15 = df  # df debe ser el nombre del DataFrame que obtuviste en la pregunta 1
    top15['PopEst'] = top15['Energy Supply'] / top15['Energy Supply per Capita']
    top15['Citable docs per Capita'] = top15['Citable documents'] / top15['PopEst']
    top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

plot9()

### Pregunta 10
Crea una nueva columna con un valor igual a 1 si el valor de % de energía renovable del país (`% Renewable`) es igual o superior de la mediana de todos los países en el top 15, o con un valor igual a 0 si el valor de % de energía renovable del país (`% Renewable`) está por debajo de la mediana.

*Esta función debería devolver una serie llamada `HighRenew` cuyo índice es el nombre del país ordenado ascendentemente según el ranking.*

In [16]:
def high_renewable():
    # Obtener los datos del top 15
    HighRenew = load_data()
    # Calcular la mediana de la columna % Renewable
    median_renewable = HighRenew['% Renewable'].median()
    # Crear la columna HighRenew con valores 1 o 0
    HighRenew['HighRenew'] = (HighRenew['% Renewable'] >= median_renewable).astype(int)
    # Ordenar los países por ranking
    HighRenew.sort_values(by='Rank', inplace=True)
    # Devolver la serie HighRenew
    return HighRenew['HighRenew']

high_renewable()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int32

### Pregunta 11
Utiliza el siguiente diccionario para agrupar los países por continente, luego crea un DataFrame que muestre el tamaño de la muestra (la cantidad de países en cada contenedor de continente) y la suma, la media y la desviación estándar para la población estimada de cada país.

```python
continent_dict  = {'China':'Asia',
                   'United States':'North America',
                   'Japan':'Asia',
                   'United Kingdom':'Europe',
                   'Russian Federation':'Europe',
                   'Canada':'North America',
                   'Germany':'Europe',
                   'India':'Asia',
                   'France':'Europe',
                   'South Korea':'Asia',
                   'Italy':'Europe',
                   'Spain':'Europe',
                   'Iran':'Asia',
                   'Australia':'Australia',
                   'Brazil':'South America'}
```

*Esta función debería devolver un DataFrame con un índice llamado Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` y columnas `['size', 'sum', 'mean', 'std']`*

In [18]:
def group_by_continent():
   
    data = load_data()
    
    # Diccionario de continentes
    continent_dict = {'China': 'Asia',
                      'United States': 'North America',
                      'Japan': 'Asia',
                      'United Kingdom': 'Europe',
                      'Russian Federation': 'Europe',
                      'Canada': 'North America',
                      'Germany': 'Europe',
                      'India': 'Asia',
                      'France': 'Europe',
                      'South Korea': 'Asia',
                      'Italy': 'Europe',
                      'Spain': 'Europe',
                      'Iran': 'Asia',
                      'Australia': 'Australia',
                      'Brazil': 'South America'}
    
   # Agregar la columna 'Continent' al DataFrame
    data['Continent'] = [continent_dict[country] for country in data.index]

    # Calcular la población estimada y agregarla al DataFrame
    data['Population'] = data['Energy Supply'] / data['Energy Supply per Capita']

    # Agrupar por continente y calcular las estadísticas
    grouped = data.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])
    
    return grouped

group_by_continent()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


### Optional

Puedes ejecutar la función `plot_optional()` para ver un ejemplo de visualización.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    top15 = df  # df debe ser el nombre del DataFrame que obtuviste en la pregunta 1
    ax = top15.plot(x='Rank', y='% Renewable', kind='scatter',
                    c=['#e41a1c', '#377eb8', '#e41a1c', '#4daf4a',
                       '#4daf4a', '#377eb8', '#4daf4a', '#e41a1c',
                       '#4daf4a', '#e41a1c', '#4daf4a', '#4daf4a',
                       '#e41a1c', '#dede00', '#ff7f00'],
                    xticks=range(1, 16), s=6 * top15['2014'] / 10 ** 10,
                    alpha=.75, figsize=[16, 6])

    for i, txt in enumerate(top15.index):
        ax.annotate(txt, [top15['Rank'][i], top15['% Renewable'][i]], ha='center')

    print(
        """Este es un ejemplo de visualización realizado para ayudarte a comprender los datos. \nEste es un gráfico de burbujas que muestra el % Renewable vs. Rank. \nEl tamaño de la burbuja corresponde al GDP (PIB) de 2014 de los países, y el color corresponde al continente.""")


plot_optional()